In [105]:
import requests
import re
import pandas as pd
import time
import random

In [106]:
position = pd.read_csv("data/raw/postesSynop.csv", sep=";")

Id = position["ID"].astype(str)
for i in range(len(Id)):
    if len(Id[i]) < 5:
        Id[i] = '0' + Id[i]

In [107]:
Id

0     07005
1     07015
2     07020
3     07027
4     07037
      ...  
57    81401
58    81405
59    81408
60    81415
61    89642
Name: ID, Length: 62, dtype: object

In [108]:
provinces = {}
failures = []
for i in range(len(Id)):
    success = False
    failTimes = 0
    while not success:
        response = requests.get("https://www.infoclimat.fr/observations-meteo/temps-reel/paris-met-center/" + Id[i] + ".html")
        text = response.text
        try:
            p = re.search('<span class="hide-on-small-only">Département&nbsp;', text).span()[1]
            provinces[int(Id[i])] = text[p] + text[p + 1]
            success = True
        except Exception:
            failTimes += 1
        time.sleep(random.random() * 5)
        if failTimes > 20:
            break
    if not success:
        failures.append(Id[i])

In [109]:
use_provinces = {}
for i in provinces.keys():
    try:
        number = int(provinces[i])
        if number <= 95:
            use_provinces[i] = number
    except Exception:
        pass